In [1]:
import pickle
from aemulusnu_massfunction.emulator_training import *
from aemulusnu_massfunction.massfunction_fitting_tinker import Tinker08Costanzi13
from aemulusnu_hmf import massfunction as hmf
# from aemulusnu_massfunction.fisher_utils import MassFuncTinker08Costanzi13

In [2]:
tinker08_hmf = Tinker08()

In [3]:
from scipy.integrate import quad

In [4]:
from tqdm import tqdm, trange
a_list_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/alist.pkl'
a_list_f = open(a_list_fname, 'rb')
a_list = pickle.load(a_list_f)
a_list_f.close()

cosmos_f = open('../data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()


In [5]:
for box in tqdm(cosmo_params):
    if('n50' not in box):
        continue
    NvM_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_NvsM.pkl'
    NvM_f = open(NvM_fname, 'rb')
    NvMs = pickle.load(NvM_f) #NvMs is a dictionary of dictionaries
    NvM_f.close()


    N_data = {}
    M_data = {}
    aux_data = {}

    vol = -1 #Mpc^3/h^3
    Mpart = -1

    for a in (a_list):
        if(a not in NvMs):
            continue
        c_data = NvMs[a]

        Ms = c_data['M'] #units of h^-1 Msolar
        N = c_data['N']
        edge_pairs = c_data['edge_pairs']
        assert(len(Ms) == len(edge_pairs))
        assert(len(Ms) == len(N))

        if(vol==-1):
            vol = c_data['vol']
        assert(vol == c_data['vol'])

        if(Mpart==-1):
            Mpart = c_data['Mpart']
        assert(Mpart == c_data['Mpart'])

        N_data[a] = []
        M_data[a] = []
        aux_data[a] = []
        for N_curr, M_curr, edge_pair in zip(N, Ms, edge_pairs):
            N_data[a] += [N_curr]
            M_data[a] += [M_curr]
            aux_data[a] += [{'a':a, 'edge_pair':edge_pair}]

    M_numerics = np.logspace(np.log10(100*Mpart), 16, 50)

    jackknife_covs_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_jackknife_covs.pkl'
    jackknife_covs_f = open(jackknife_covs_fname, 'rb')
    jackknife = pickle.load(jackknife_covs_f)
    jackknife_covs_f.close()

    jack_covs = {a:jackknife[a][1] for a in N_data}

    # Compute the weighted covariance matrix incorporating jackknife and poisson
    weighted_cov = {a: jack_covs[a] for a in jack_covs}

    param_names = ['d','e','f','g']

    h = cosmo_params[box]['H0']/100

    cosmology = hmf.cosmology(cosmo_params[box])
    for a in (N_data):
        yerr = np.sqrt(np.diagonal(weighted_cov[a]))
        c_data = NvMs[a]

        Ms = M_data[a]
        N = N_data[a]
        edge_pairs = c_data['edge_pairs']

        edges = [edge[0] for edge in edge_pairs]
        edges += [edge_pairs[-1][1]]

        #shade in 1% and 10% error region
        edges = np.array(edges)



        y1 = 0.1*np.ones_like(N)
        y1 = np.append(y1, y1[-1])
        y1 = np.append(y1[0], y1)

        y2 = -0.1*np.ones_like(N)
        y2 = np.append(y2, y2[-1])
        y2 = np.append(y2[0], y2)

        c_Ms = np.append(Ms, edges[-1])
        c_Ms = np.append(edges[0], c_Ms)

        y1 = 0.01*np.ones_like(N)
        y1 = np.append(y1, y1[-1])
        y1 = np.append(y1[0], y1)

        y2 = -0.01*np.ones_like(N)
        y2 = np.append(y2, y2[-1])
        y2 = np.append(y2[0], y2)



        dM = np.array([edges[1]-edges[0] for edges in edge_pairs])
        #tinker08 + costanzi13

        mass_function = tinker08_hmf

        f_dNdM_MCMC =  lambda M:mass_function(cosmology, M, a)*vol # h / Msun
        tinker_eval_MCMC = np.array([quad(f_dNdM_MCMC, edge[0],  edge[1], epsabs=0, epsrel=1e-3)[0] for edge in edge_pairs])
        with open("/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/%s_%.2f_NvMtinker08costanzi13_loo_output_debug.pkl"%(box, a), "wb") as f:
            pickle.dump({'Ms':Ms, 'tinker_eval':tinker_eval_MCMC, 'N':N, 'edges':edges}, f)


100%|██████████| 150/150 [14:43<00:00,  5.89s/it]
